<a href="https://colab.research.google.com/github/lyuyanyii/text-classification/blob/main/newsgroup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

print(np.random.normal(loc = 0, scale = 1, size = (10, 1)))


import os

os.system('wget https://github.com/lil-lab/lm-class/raw/refs/heads/main/assignments/a1/starter-repo/data/newsgroups/train/train_data.csv')


[[-1.36371682]
 [-0.13497666]
 [-1.06752997]
 [ 0.65696306]
 [ 0.87770999]
 [ 1.47520152]
 [ 0.83228965]
 [ 0.24712302]
 [ 0.07981165]
 [-1.22591496]]


In [ ]:
os.system('wget https://github.com/lil-lab/lm-class/raw/refs/heads/main/assignments/a1/starter-repo/data/newsgroups/train/train_labels.csv')


0

In [ ]:
import csv
import string

D = 128

def data_process(raw_text : string):
    text = raw_text.lower()
    raw_word_list = text.split()
    word_list = []
    for raw_word in raw_word_list:
        word = ''.join([i for i in raw_word if i.isalpha()])
        if word != '':
            word_list.append(word)
    return word_list

def bag_of_words_embedding(word_vector_dict : dict, word_list : list):
    vector_list = []

    for word in word_list:
        if word not in word_vector_dict:
            word_vector_dict[word] = np.random.normal(loc = 0, scale = 0.5, size = (D, 1))

        embedding = word_vector_dict[word]
        vector_list.append(embedding)

    bag_of_words = np.concatenate(vector_list, axis = 1).mean(axis = 1)
    return bag_of_words

train_data = []
word_vector_dict = {}

with open('train_data.csv', newline = '') as file:
    csv_file = csv.DictReader(file)
    for line in csv_file:
        word_list = data_process(line['text'])

        train_data.append(bag_of_words_embedding(word_vector_dict, word_list))

print(len(train_data))

9051


In [ ]:
train_labels = []
label_index_map = {}

num_class = 0

with open('train_labels.csv', newline = '') as file:
    csv_file = csv.DictReader(file)
    for line in csv_file:
        label = line['newsgroup']

        if label not in label_index_map:
            label_index_map[label] = num_class
            num_class += 1

        index = label_index_map[label]
        train_labels.append(index)

print(num_class)
print(len(train_labels))

20
9051


In [ ]:
train_ratio = 0.8
dataset_size = len(train_data)

valid_data = train_data[ int(dataset_size * train_ratio) : ]
valid_labels = train_labels[ int(dataset_size * train_ratio) : ]

train_data = train_data[ : int(dataset_size * train_ratio) ]
train_labels = train_labels[ : int(dataset_size * train_ratio) ]

print(len(train_data))
print(len(valid_data))

7240
1811


In [ ]:
class Linear:
    def __init__(self, in_dim, out_dim):
        self.w = np.random.normal(loc = 0, scale = 1 / in_dim**0.5, size = (out_dim, in_dim))
        self.b = np.random.normal(loc = 0, scale = 1 / in_dim**0.5, size = (out_dim, 1))
        self.grad_w = np.zeros((out_dim, in_dim))
        self.grad_b = np.zeros((out_dim, 1))

    def forward(self, x):
        self.x = x
        self.out = np.matmul(self.w, x) + self.b
        return self.out

    def back_prop(self, grad_out):
        self.grad_b = grad_out.mean(axis = 0)
        self.grad_w = np.matmul(grad_out, self.x.transpose(0, 2, 1)).mean(axis = 0)
        self.grad_x = np.matmul(self.w.transpose((1, 0)), grad_out)
        return self.grad_x

    def update(self, lr):
        self.w += self.grad_w * lr
        self.b += self.grad_b * lr


[[0.01965774 0.1078472 ]
 [0.43081159 0.2941039 ]
 [0.3580904  0.64021895]]
[[-0.90678279]
 [-0.13409614]
 [-0.61950386]]


In [ ]:
class Relu:
    def __init__(self):
        pass

    def forward(self, x):
        self.mask = (x >= 0)
        return x * self.mask

    def back_prop(self, grad_y):
        return grad_y * self.mask


In [ ]:
class Softmax:
    def __init__(self):
        pass

    def forward(self, x):
        self.x = x
        exp_x = np.exp(x)
        d1, d2, d3 = x.shape
        self.y = exp_x / exp_x.sum(axis = 1).reshape(d1, 1, 1)
        return self.y

    def back_prop(self, grad_y):
        (d1, d2, d3) = grad_y.shape
        # print((np.identity(d2) - self.y).shape)
        # print((self.y.transpose(0, 2, 1)* (np.identity(d2, ) - self.y)).shape)
        # print(self.y.transpose(0, 2, 1))
        # print(self.y.transpose(0, 2, 1)* (np.identity(d2, ) - self.y))
        grad_x = np.matmul( (self.y.transpose(0, 2, 1) * (np.identity(d2) - self.y)), grad_y )
        return grad_x

"""
x = np.arange(0, 3)
x = x.reshape((1, 3, 1))
sft_layer = Softmax()
y = sft_layer.forward(x)
print(y)
grad_y = np.array([1,0,0])
grad_y = grad_y.reshape((1, 3, 1))
grad_x = sft_layer.back_prop(grad_y)
print(grad_x)
"""

'\nx = np.arange(0, 3)\nx = x.reshape((1, 3, 1))\nsft_layer = Softmax()\ny = sft_layer.forward(x)\nprint(y)\ngrad_y = np.array([1,0,0])\ngrad_y = grad_y.reshape((1, 3, 1))\ngrad_x = sft_layer.back_prop(grad_y)\nprint(grad_x)\n'

In [ ]:
batch_size = 10
depth = 3

network = []
for i in range(depth - 1):
    network.append((Linear(D, D), Relu()))
network.append((Linear(D, num_class), Softmax()))

def train(network, data, labels, lr):
    x = np.vstack(data)
    (d1, d2) = x.shape
    x = x.reshape(d1, d2, 1)
    for (layer, f_layer) in network:
        x = f_layer.forward(layer.forward(x))

    # loss1 = - np.log(np.vstack([x[i][labels[i]][0] for i in range(d1)]))
    d1, d2, d3 = x.shape
    loss = - np.log(x[range(d1), labels, 0]).mean()
    grad_x = np.zeros((d1, d2, d3))
    for i in range(d1):
        grad_x[i, labels[i], 0] = 1/x[i, labels[i], 0]

    for (layer, f_layer) in reversed(network):
        grad_y1 = f_layer.back_prop(grad_x)
        grad_y2 = layer.back_prop(grad_y1)
        grad_x = grad_y2
        layer.update(lr)

    return loss

def eval(network, data):
    x = np.vstack(data)
    (d1, d2) = x.shape
    x = x.reshape(d1, d2, 1)
    for (layer, f_layer) in network:
        x = f_layer.forward(layer.forward(x))

    d1, d2, d3 = x.shape
    x = x.reshape(d1, d2)
    idx = np.argmax(x, axis = 1)
    return idx

num_epochs = 30

for i in range(num_epochs):
    loss = 0
    print('---- Epoch ', i, '----')
    for j in range(len(train_data) // 10):
        start = j * 10
        end = min(start + 10, len(train_data))
        loss = train(network, train_data[start:end], train_labels[start:end], lr = 0.1)
    print('Loss = ', loss)

    pred = eval(network, valid_data)
    acc = np.array(pred == valid_labels).mean()
    print('Validation Accurary = ', acc)


---- Epoch  0 ----
Loss =  2.865137022134048
Validation Accurary =  0.10877967973495306
---- Epoch  1 ----
Loss =  2.334840964357313
Validation Accurary =  0.12534511319712865
---- Epoch  2 ----
Loss =  2.0913923632225666
Validation Accurary =  0.18056322473771397
---- Epoch  3 ----
Loss =  1.958881150471436
Validation Accurary =  0.22418553285477635
---- Epoch  4 ----
Loss =  1.8752006602857514
Validation Accurary =  0.26780784097183874
---- Epoch  5 ----
Loss =  1.7717380361872017
Validation Accurary =  0.2992821645499724
---- Epoch  6 ----
Loss =  1.7076223340537815
Validation Accurary =  0.315847598012148
---- Epoch  7 ----
Loss =  1.6897743487110666
Validation Accurary =  0.34732192159028163
---- Epoch  8 ----
Loss =  1.6797477521370012
Validation Accurary =  0.3556046383213694
---- Epoch  9 ----
Loss =  1.6576702734747386
Validation Accurary =  0.36112644947542794
---- Epoch  10 ----
Loss =  1.6235430065094465
Validation Accurary =  0.36664826062948647
---- Epoch  11 ----
Loss = 